In [ ]:
#extract filenames from /content/xml/epidoc and write them into a csv file
# NB! REPLACE DIRECTORY PATHS AS APPROPRIATE

import glob
import csv

mylist = [f for f in glob.glob("/Users/simonastoyanova/Desktop/ircyr-efes/webapps/ROOT/content/xml/epidoc/*.xml")]
mylist.sort()
#print(mylist)


with open("/Users/simonastoyanova/Desktop/ircyr-test/epidoc-dir.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in mylist:
        wr.writerow([word])


In [ ]:
#check filename and content of <idno type="filename"> for consistency
# NB! REPLACE DIRECTORY PATHS AS APPROPRIATE

import re

with open("epidoc-dir.csv", 'r') as listfiles:
    lf = listfiles.readlines()
   # print(lf)
    for x in lf:
        x = x.rstrip()
        if len(x.split('.')[0]) != 6:
            print("ERROR in " + x + " malformed filename!")
        if x !="Filenames":
            #print(x)
            #fpath = '/Users/simonastoyanova/Desktop/ircyr-test/input/' + x     #ADD EXCEPTION HERE
            fpath = '/Users/simonastoyanova/Desktop/ircyr-efes/webapps/ROOT/content/xml/epidoc/' + x     #ADD EXCEPTION HERE
            infile = open(fpath,"r")
            contents = infile.read()
            idnos = re.findall(r'<idno type="filename">(.*?)</idno>', contents)
            if len(idnos) > 1:
                print("ERROR in " + x + " too many idnos!")
            elif len(idnos) == 0:
                print("ERROR in " + x + " no idno!")
            if len(idnos[0]) != 6:
                print("ERROR in " + x + " malformed idno ("+idnos[0]+") !")
            if x.split('.')[0] != idnos[0]:
                print("ERROR in " + x + " idno ("+idnos[0]+") doesn't match filename!")

                

In [ ]:
#replace old filenames with new ones
#add new <idno> with new filename
#replace all references to old filenames with new filenames
# NB! REPLACE DIRECTORY PATHS AS APPROPRIATE

import os
import csv
import shutil
import glob
import re

allreplace = {}
with open("all-replacements.csv", "r") as file:
    content = csv.reader(file)
    for row in content:
        allreplace[row[0]] = row[1]

def renamefiles():
    '''replaces old filenames with new ones, taken from a csv file'''
    inpath = "/Users/simonastoyanova/Desktop/ircyr-test/input/"
    outpath = "/Users/simonastoyanova/Desktop/ircyr-test/newfiles/"
    for filename in os.listdir(inpath):
        if filename.endswith('.xml'):
            old_name = filename.split('.')
            shutil.copy(inpath+filename,outpath+allreplace[old_name[0]]+'.xml')
    print('all files renamed')

def replaces():
    '''replaces the content of <idno type="filename"> with the new numbers;
    keeps the old number in <idno type="ircyr2012">'''
    for file in glob.glob("/Users/simonastoyanova/Desktop/ircyr-test/newfiles/*.xml"):
        infile = open(file, "r")
        contents = infile.read()
        
        oldid = re.findall('<idno type="filename">(.*?)</idno>', contents)[0]
        newid = allreplace[oldid]

        oldstr = '<idno type="filename">' + oldid + '</idno>'
        newstr = '<idno type="filename">' + newid + '</idno>\n            <idno type="ircyr2012">' + oldid + '</idno>'

        contents = contents.replace(oldstr, newstr)
        
        allrefs = re.findall('<ref type="inscription">(.*?)</ref>', contents)
        for x in allrefs:
            if x in allreplace.keys():
                newnum = allreplace[x]
                oldref = '<ref type="inscription">' + x + '</ref>'
                newref = '<ref type="inscription" n="' + x + '">' + newnum + '</ref>'
                contents = contents.replace(oldref, newref)
            else:
                print('ERROR ' + x + ' referenced in ' + newid + ' but not in allreplace.csv!')
    
        outfile = open(file, "w")
        outfile.write(contents)
        outfile.close()
    print('all idnos & refs replaced, files overwritten')


renamefiles()
replaces()